## Implied Volatility

Implied volatility (IV) is one of the most important parameter in options pricing. IV is determined by the current market price of option contracts on a particular underlying asset. IV is commonly represented as a percentage that indicates the annualized expected one standard deviation range for the underlying asset implied from the option prices. 

IV  $\sigma_{imp}$ is the volatility value $\sigma$ that makes the Black Scholes value of the option equal to the traded price of the option. In the Black-Scholes model, volatility is the only parameter that can't be directly observed. All other parameters can be determined through market data and this parameter is determined by a numerical optimization technique given the Black-Scholes model.

**Import Required Libraries**

In [1]:
# Import libraries
from numpy import *
import pandas as pd
from tabulate import tabulate
import matplotlib.pyplot as plt
from datetime import date, datetime, timedelta

from quantmod.derivatives import OptionData 
from quantmod.models import OptionInputs, BlackScholesOptionPricing


In [2]:
def newton_iv(spot, strike, rate, dte, callprice=None, putprice=None):

    x0 =1.                                      # initial guess, type float
    h = 0.001                                   # step size 
    tolerance = 1e-7                            # 7-digit accuracy is desired
    epsilon = 1e-14                             # do not divide by a number smaller than this, some kind of error / floor
    maxiter = 200                               # maximum number of iterations to execute
    
    # function whose root we are trying to find
    # f(x) = Black Scholes Call price - Market Price - defining the f(x) here
    if callprice:
        f = lambda x: BlackScholesOptionPricing(OptionInputs(spot=spot, strike=strike, rate=rate, ttm=dte, volatility=x)).call_price - callprice
    if putprice:
        f = lambda x: BlackScholesOptionPricing(OptionInputs(spot=spot, strike=strike, rate=rate, ttm=dte, volatility=x)).putPrice - putprice
        
    for i in range(maxiter):
        y = f(x0)                               # starting with initial guess
        yprime = (f(x0+h) - f(x0-h))/(2*h)      # central difference, the derivative of the function
        
        if abs(yprime)<epsilon:                 # stop if the denominator is too small
            break                              
        x1 = x0 - y/yprime                      # perform Newton's computation
        
        if (abs(x1-x0) <= tolerance*abs(x1)):   # stop when the result is within the desired tolerance
            break
        x0=x1                                   # update x0 to start the process again
        
    return x1                                   # x1 is a solution within tolerance and maximum number of iterations

In [3]:
# newton iv
newton_iv(100,100,0.05,1, callprice=8)

0.13377582588953527

In [4]:
# verfiy the output
BlackScholesOptionPricing(OptionInputs(spot=100, strike=100, rate=0.05, ttm=1, volatility=0.13377582588953527)).call_price

7.9999999999999645

### Bisection Method
The bisection method is considered to be one of the simplest and robust root finding algorithm. 

Suppose, we know the two points of an interval $a$ and $b$, where $a<b$ and $f(a)<0$ and $f(b)>0$ lie along the continuous function and the mid-point of this interval $c = \frac{a+b}{2}$, then we can evaluate the value as $f(c)$.

Iteratively, we replace $c$ as either $a$ or $b$, thereby shortening the interval to find the root. If $f(c) = 0$ or within acceptable value, we have a root. Bisection methods are stable and guarantee to converge. As it does not require knowledge of the derivative, it takes more computational time.

In [5]:
# Bisection Method
def bisection_iv(spot, strike, rate, dte, callprice=None, putprice=None, high=500.0, low=0.0):
    
    # this is market price
    if callprice:
        market_price = callprice
    if putprice and not callprice:
        market_price = putprice
        
    tolerance = 1e-7
        
    for i in range(1000):
        mid = (high + low) / 2             # c= (a+b)/2
        if mid < tolerance:
            mid = tolerance
            
        if callprice:
            blackscholes_price = BlackScholesOptionPricing(OptionInputs(spot = spot,strike = strike,rate = rate,ttm = dte,volatility = mid,)).call_price
        if putprice:
            blackscholes_price = BlackScholesOptionPricing(OptionInputs(spot = spot,strike = strike,rate = rate,ttm = dte,volatility = mid,)).put_price
        
        if round(blackscholes_price,7) == market_price:
            break
        elif blackscholes_price > market_price: 
            high = mid                      # replace c with b | b = c
        elif blackscholes_price < market_price: 
            low = mid                       # replace c with a | a = c
    
    return mid

In [6]:
# bisection iv
bisection_iv(100,100,0.02,1,callprice=8.)

0.17657213902566582

## BS Implied Volatility
Let's now update our Blackscholes' class to incorporate implied volatility.

In [7]:
# Initialize option
option = BlackScholesOptionPricing(OptionInputs(spot = 100, strike = 100, rate = 0.02, ttm = 1, volatility = 0.2, callprice=8))

header = ['Option Price', 'Delta', 'Gamma', 'Theta', 'Vega', 'Rho', 'IV']
table = [[option.call_price, option.call_delta, option.gamma, option.call_theta, option.vega, option.call_rho, option.impvol]]

print(tabulate(table,header))

  Option Price    Delta      Gamma      Theta      Vega       Rho        IV
--------------  -------  ---------  ---------  --------  --------  --------
       8.91604  0.57926  0.0195521  -0.013399  0.391043  0.490099  0.176572


In [8]:
# Initialize option
option = BlackScholesOptionPricing(OptionInputs(spot = 100, strike = 100, rate = 0.02, ttm = 1, volatility = 0.2, putprice=8))

header = ['Option Price', 'Delta', 'Gamma', 'Theta', 'Vega', 'Rho', 'IV']
table = [[option.put_price, option.put_delta, option.gamma, option.put_theta, option.vega, option.put_rho, option.impvol]]

print(tabulate(table,header))

  Option Price     Delta      Gamma        Theta      Vega        Rho        IV
--------------  --------  ---------  -----------  --------  ---------  --------
        6.9359  -0.42074  0.0195521  -0.00802802  0.391043  -0.490099  0.227215


## Nifty Option
We will extend the option analysis carried out from Lab 5 to build IV Skew function. 

`Note`: You will need to update the quantmod library to version 0.0.6 to use Indian options data.

### Query Option Chain

In [9]:
# Instantiate the Option Data
opt = OptionData("NIFTY", "24-Apr-2025")
df = opt.get_call_option_data

# query strikes between 23500 and 24500
df = df.query('strikePrice>=23500 and strikePrice<=24500').reset_index(drop=True)
df.head(2)

,strikePrice,expiryDate,underlying,identifier,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue
0,23500,24-Apr-2025,NIFTY,OPTIDXNIFTY24-04-2025CE23500.00,61855,-5787,-8.555335,27703,20.85,569.45,139.1,32.322528,850500,49650,75,567.85,225,569.15,24043.8
1,23550,24-Apr-2025,NIFTY,OPTIDXNIFTY24-04-2025CE23550.00,1894,-795,-29.564894,2923,20.82,523.50,129.8,32.969266,104100,35775,75,520.50,75,523.10,24043.8


### IV Skew
Create a user defined function to calculate the IV Skew

In [10]:
def ivskew(spot, rate, valuation, expiration, frame):
    
    dte = (pd.to_datetime(expiration+' 15:30:00') - pd.to_datetime(valuation)) / timedelta(days=365)

    # Instantiate BS Pricing Engine from quantmod and Derive Implied Volatiliy
    for i in range(len(frame)):
        nifty = BlackScholesOptionPricing(
            OptionInputs(
                spot = spot,
                strike = frame['strikePrice'].iloc[i], 
                rate = rate,
                ttm = dte,
                volatility = 0.20,
                callprice = frame['lastPrice'].iloc[i]
                )
            )
            
        frame.loc[i, 'IV'] = nifty.impvol
    frame.set_index('strikePrice', inplace=True)
        
    return frame['IV']

In [11]:
# call the iv skew function
ivskew(22850, 0,'25-Mar-2025', '24-Apr-2025', df).head(2)

strikePrice
23500    0.319453
23550    0.308411
Name: IV, dtype: float64

In [ ]:
# Set expiry list
expiry_list = ['09-Apr-2025' , '24-Apr-2025']

# Initialise empty dataframe
call_price = pd.DataFrame()
call_vols = pd.DataFrame()

for expiry in expiry_list:
    
    opt = OptionData("NIFTY", expiry)
    df = (opt.get_call_option_data
          .query('strikePrice>=23500 and strikePrice<=24500').reset_index(drop=True)
    )
            
    call_price[expiry] = df['lastPrice']
    call_vols[expiry] = ivskew(22850, 0, '25-Mar-2025', expiry, df)


In [ ]:
# Verify Call IV
call_vols

### Visualize

In [ ]:
# plot IV Skew & Term Structure
plt.plot(call_vols.index, call_vols)
plt.title('IV Skew & Term Structure')
plt.xlabel('Strike Price')
plt.ylabel('Implied Volatility')
plt.legend(call_vols.columns)
plt.show()